# Example 08 - Optuna with Wideband and YOLO
This notebook showcases Optuna hyperparameter tuning a YOLOv8 model with Wideband.

---

## Import Libraries

In [2]:
# Packages for Optuna
from torchsig.utils.optuna.tuner import YoloOptunaOptimizer
import copy

In [3]:
# Packages Imports for Training
from ultralytics import YOLO
from torchsig.utils.yolo_train import Yolo_Trainer
from torchsig.datasets.datamodules import WidebandDataModule
from torchsig.transforms.transforms import Compose, Spectrogram, Normalize, SpectrogramImage
from torchsig.transforms.target_transforms import DescToBBoxFamilyDict
from torchsig.datasets.signal_classes import torchsig_signals
import torch
import yaml
import numpy as np
from torchsig.image_datasets.plotting.plotting import plot_yolo_datum


-----------------------------
## Check or Generate the Wideband Dataset
To generate the Wideband dataset, several parameters are given to the imported `WidebandDataModule` class. These paramters are:
- `root` ~ A string to specify the root directory of where to generate and/or read an existing Wideband dataset
- `train` ~ A boolean to specify if the Wideband dataset should be the training (True) or validation (False) sets
- `qa` - A boolean to specify whether to generate a small subset of Wideband (True), or the full dataset (False), default is True
- `impaired` ~ A boolean to specify if the Wideband dataset should be the clean version or the impaired version
- `transform` ~ Optionally, pass in any data transforms here if the dataset will be used in an ML training pipeline. Note: these transforms are not called during the dataset generation. The static saved dataset will always be in IQ format. The transform is only called when retrieving data examples.
- `target_transform` ~ Optionally, pass in any target transforms here if the dataset will be used in an ML training pipeline. Note: these target transforms are not called during the dataset generation. The static saved dataset will always be saved as tuples in the LMDB dataset. The target transform is only called when retrieving data examples.

A combination of the `train` and the `impaired` booleans determines which of the four (4) distinct Wideband datasets will be instantiated:
| `impaired` | `qa` | Result |
| ---------- | ---- | ------- |
| `False` | `False` | Clean datasets of train=250k examples and val=25k examples |
| `False` | `True` | Clean datasets of train=250 examples and val=250 examples |
| `True` | `False` | Impaired datasets of train=250k examples and val=25k examples |
| `True` | `True` | Impaired datasets of train=250 examples and val=250 examples |

The final option of the impaired validation set is the dataset to be used when reporting any results with the official Wideband dataset.

In [4]:
# Generate Wideband DataModule
root = "./datasets/wideband"
impaired = True
qa = True
fft_size = 512
class_list = torchsig_signals.class_list
num_classes = len(class_list)
batch_size = 1

transform = Compose([    
])

target_transform = Compose([
    DescToBBoxFamilyDict()
])

datamodule = WidebandDataModule(
    root=root,
    impaired=impaired,
    qa=qa,
    fft_size=fft_size,
    num_classes=num_classes,
    transform=transform,
    target_transform=target_transform,
    batch_size=batch_size
)

datamodule.prepare_data()
datamodule.setup("fit")

wideband_train = datamodule.train

# Retrieve a sample and print out information
idx = np.random.randint(len(wideband_train))
data, label = wideband_train[idx]
print("Dataset length: {}".format(len(wideband_train)))
print("Data shape: {}".format(data.shape))
print("Label: {}".format(label))

Using WidebandImpairedTrainQAConfig for train.
Dataset already exists in ./datasets/wideband/wideband_impaired_train. Not regenerating
Using WidebandImpairedValQAConfig for val.
Dataset already exists in ./datasets/wideband/wideband_impaired_val. Not regenerating
Dataset length: 250
Data shape: (262144,)
Label: {'labels': tensor([5, 5, 5, 9]), 'boxes': tensor([[0.0950, 0.1786, 0.1900, 0.0491],
        [0.5282, 0.5223, 0.1900, 0.0491],
        [0.9331, 0.2768, 0.1337, 0.0491],
        [0.5000, 0.5031, 1.0000, 0.1219]])}


## Prepare YOLO trainer and Model
Next, the datasets are rewritten to disk that is Ultralytics YOLO compatible. See [Ultralytics: Train Custom Data - Organize Directories](https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#23-organize-directories) to learn more. 

Additionally, create a yaml file for dataset configuration. See [Ultralytics: Train Custom Data - Create dataset.yaml](https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#21-create-datasetyaml)

Download desired YOLO model from [Ultralytics Models](https://docs.ultralytics.com/models/). We will use YOLOv8, specifically `yolov8n.pt`

---

### Explanation of the `overrides` Dictionary

The `overrides` dictionary is used to customize the settings for the Ultralytics YOLO trainer by specifying specific values that override the default configurations. The dictionary is imported from `wbdata.yaml`. However, you can customize in the notebook. 

Example:

```python
overrides = {'model': 'yolov8n.pt', 'epochs': 100, 'data': '08_example.yaml', 'device': 0, 'imgsz': 512, 'single_cls': True}
```
A .yaml is necessary for training. Look at `08_yolo_optuna.yaml` in the examples directory. It will contain the path to your torchsig data.


### Dataset Location Warning

There must exist a datasets directory at `/path/to/torchsig/datasets`.

This example assumes that you have generated `train` and `val` lmdb wideband datasets at `./datasets/wideband/`

You can also specify an absolute path to your dataset in `08_yolo_optuna.yaml`.

In [5]:
# define dataset variables for yaml file
config_name = "08_yolo_optuna.yaml"
classes = {v: k for v, k in enumerate(class_list)}
yolo_root = "./wideband/" # train/val images (relative to './datasets``

# define overrides
overrides = dict(
    model = "yolov8n.pt",
    project = "yolo",
    name = "08_example",
    epochs = 10,
    imgsz = 512,
    data = config_name,
    device = 0 if torch.cuda.is_available() else "cpu",
    single_cls = True,
    batch = 32,
    workers = 8

)

# create yaml file for trainer
yolo_config = dict(
    overrides = overrides,
    train = yolo_root,
    val = yolo_root,
    nc = num_classes,
    names = classes
)

with open(config_name, 'w+') as file:
    yaml.dump(yolo_config, file, default_flow_style=False)
print(f"Creating experiment -> {overrides['name']}")

Creating experiment -> 08_example


## Run Optuna
Now we can start using optuna to tune the following hyperparameters in YOLO:
- `lr0` - Initial learning rate.
- `cos_lr` - Toggle cosine learning rate scheduler.
- `optimizer` - Choose optimizer (SGD, Adam, AdamW)
- `freeze` - Freeze the first N layers of the model.
- `imgsz` - Target image size for training.

Within the optuna optimizer,`n_trials` determines how many trials to run, while `epochs` is how many epochs are run per trial.

See [Ultralytics Train Settings](https://docs.ultralytics.com/usage/cfg/#train-settings) for more hyperparameters.

In [6]:
opt = YoloOptunaOptimizer(overrides, n_trials=2, epochs=1)
study, best_params = opt.run_optimization()

overrides_optimized = opt.get_optimized_overrides()

[I 2025-01-14 13:41:59,595] A new study created in memory with name: Test


Ultralytics 8.3.3 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15981MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=08_yolo_optuna.yaml, epochs=1, time=None, patience=100, batch=32, imgsz=256, save=False, save_period=-1, cache=False, device=0, workers=1, project=yolo, name=08_example4, exist_ok=False, pretrained=True, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=3, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

2025/01/14 13:42:03 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
2025/01/14 13:42:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for pytorch_lightning.
2025/01/14 13:42:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/01/14 13:42:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


MLflow: logging run_id(a33cc226e56745d08af344e88e9689fd) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 1 dataloader workers
Logging results to yolo/08_example4
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1     0.648G      4.817      6.063      2.681        251        256: 100%|██████████| 8/8 [00:12<00:00,  1.53s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.84it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00, 14.57it/s]

                   all        250       1116          0          0          0          0


Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to yolo/08_example5
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


[I 2025-01-14 13:42:46,319] Trial 1 finished with value: 0.0 and parameters: {'lr': 0.0029050204520835476, 'cos_lr': True, 'freeze': 0, 'imgsz_power2': 6, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.0.




Number of finished trials: 2
Best trial:
  Value: 0.0
  Params: 
    lr: 0.0072616937413024565
    cos_lr: True
    freeze: 3
    imgsz_power2: 8
    optimizer: SGD


## Train YOLO with Optimized Hyperparameters
Train YOLO. See [Ultralytics Train](https://docs.ultralytics.com/modes/train/#train-settings) for training hyperparameter options.

---

In [7]:
trainer = Yolo_Trainer(overrides=overrides_optimized)

trainer.train()

Ultralytics 8.3.3 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15981MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=08_yolo_optuna.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=yolo, name=08_example6, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=3, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

2025/01/14 13:42:52 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
2025/01/14 13:42:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for pytorch_lightning.
2025/01/14 13:42:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/01/14 13:42:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


MLflow: logging run_id(6c0611907a5a4e2eb95612da2a2927f0) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to yolo/08_example6
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10     0.656G      5.113      6.648      3.252         87        256: 100%|██████████| 8/8 [00:01<00:00,  7.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00, 15.23it/s]

                   all        250       1116          0          0          0          0


...

      10/10     0.648G      2.139      3.499      1.303         78        256: 100%|██████████| 8/8 [00:00<00:00, 20.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00, 16.48it/s]

                   all        250       1116          1   0.000896        0.5       0.35



10 epochs completed in 0.007 hours.
Optimizer stripped from yolo/08_example6/weights/last.pt, 5.6MB
Optimizer stripped from yolo/08_example6/weights/best.pt, 5.6MB

Validating yolo/08_example6/weights/best.pt...
Ultralytics 8.3.3 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15981MiB)
Model summary (fused): 186 layers, 2,696,458 parameters, 0 gradients, 6.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00, 15.53it/s]


                   all        250       1116          1   0.000896        0.5       0.35
                   ook        250       1116          1   0.000896        0.5       0.35
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to yolo/08_example6
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


## Evaluation
Check model performance from training. From here, you can use the trained model to test on prepared data (numpy image arrays of spectrograms)

Will load example from Torchsig

model_path is path to best.pt from your training session. Path is printed at the end of training.

---

## Generate and Instantiate WBSig53 Test Dataset
After generating the WBSig53 dataset (see `03_example_Wideband_dataset.ipynb`), we can instantiate it with the needed transforms. Change `root` to test dataset path.

---

In [8]:
test_path = './datasets/wideband_test' #Should differ from your training dataset

transform = Compose([
    Spectrogram(nperseg=512, noverlap=0, nfft=512, mode='psd'),
    Normalize(norm=np.inf, flatten=True),
    SpectrogramImage(), 
    ])

test_data = WidebandDataModule(
    root=test_path,
    impaired=impaired,
    qa=qa,
    fft_size=fft_size,
    num_classes=num_classes,
    transform=transform,
    target_transform=None,
    batch_size=batch_size
)

test_data.prepare_data()
test_data.setup("fit")

wideband_test = test_data.train

# Retrieve a sample and print out information
idx = np.random.randint(len(wideband_test))
data, label = wideband_test[idx]
print("Dataset length: {}".format(len(wideband_test)))
print("Data shape: {}".format(data.shape))

samples = []
labels = []
for i in range(10):
    idx = np.random.randint(len(wideband_test))
    sample, label = wideband_test[idx]
    lb = [l['class_name'] for l in label]
    samples.append(sample)
    labels.append(lb)

Using WidebandImpairedTrainQAConfig for train.
Dataset already exists in ./datasets/wideband_test/wideband_impaired_train. Not regenerating
Using WidebandImpairedValQAConfig for val.
Dataset already exists in ./datasets/wideband_test/wideband_impaired_val. Not regenerating
Dataset length: 250
Data shape: (512, 512, 3)


### Load model 
The model path is printed after training. Use the best.pt weights

In [9]:
%matplotlib inline

In [10]:
model = YOLO(trainer.best)
# Inference will be saved to path printed after predict. 
results = model.predict(samples, save=True, imgsz=512, conf=0.5)


0: 512x512 (no detections), 3.4ms
1: 512x512 (no detections), 3.4ms
2: 512x512 (no detections), 3.4ms
3: 512x512 (no detections), 3.4ms
4: 512x512 (no detections), 3.4ms
5: 512x512 (no detections), 3.4ms
6: 512x512 (no detections), 3.4ms
7: 512x512 (no detections), 3.4ms
8: 512x512 (no detections), 3.4ms
9: 512x512 (no detections), 3.4ms
Speed: 1.0ms preprocess, 3.4ms inference, 0.1ms postprocess per image at shape (1, 3, 512, 512)
Results saved to runs/detect/predict4


In [13]:
# Process results list
for y, result in enumerate(results):
    boxes = result.boxes  # Boxes object for bounding box outputs
    probs = result.probs  # Probs object for classification outputs
    print(f'Actual Labels -> {labels[y]}')
    plot_yolo_datum(results)
    # result.show()  # display to screen

Actual Labels -> ['16msk', 'ofdm-600', 'ofdm-256']


AttributeError: 'list' object has no attribute 'img'